## Chapter 6 - Decision Trees

### Setup

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp

from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

### Solutions

**1** What is the approximate depth of a decision tree trained (without restrictions) on a training set with one million instances?

A decision tree trained without restrictions will keep creating splits until all leaves are pure, which leads to $m$ total nodes, where $m$ is the number of training instances. \
Given that for depth $k$ there will be $2^k$ nodes, if we set $m = 2^k$ and take logarithms of both sides we get $k = log_2(m)$, which for one million observations leads to a depth of about ~20. 

### 7: Train and fine-tune a Decision Tree for the moons dataset by following these steps:

a. Use `make_moons(n_samples=10000, noise=0.4)` to generate a moons dataset.

In [ ]:
X_moons, y_moons = make_moons(n_samples=10000, noise=0.4, random_state=0)

In [ ]:
# Visualise the moons dataset
plt.scatter(X_moons[:,0], X_moons[:,1], c=y_moons, cmap='Paired', s=2)
plt.title('Moons dataset')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

b. Use train_test_split() to split the dataset into a training set
and a test set.

In [ ]:
X_moons_train, X_moons_test, y_moons_train, y_moons_test = train_test_split(X_moons, y_moons,
                                             test_size=0.2, random_state=0)

c. Use grid search with cross-validation (with the help of the `GridSearchCV` class) to find good hyperparameter values for a `DecisionTreeClassifier`. \
Hint: try various values for `max_leaf_nodes`.

In [ ]:
params = {'max_leaf_nodes': list(range(2, 100)), 
          'min_samples_split': [2, 3, 4],
          'max_depth': list(range(1,7))}
grid_dt = GridSearchCV(DecisionTreeClassifier(random_state=0),
                          param_grid=params,
                            verbose=1, n_jobs=-1)
grid_dt.fit(X_moons_train, y_moons_train)

In [ ]:
grid_dt.best_estimator_

d. Train it on the full training set using these hyperparameters, and measure your model's performance on the test set. You should get roughly 85% to 87% accuracy.

In [ ]:
# Create a new DT model with the best hyperparameters
dt = DecisionTreeClassifier(**grid_dt.best_params_, random_state=0)

# Train the model on full training set
dt.fit(X_moons_train, y_moons_train)

# Evaluate the model on the test set
dt.score(X_moons_test, y_moons_test)

# Or use more different method
y_moons_pred = dt.predict(X_moons_test)
accuracy = accuracy_score(y_moons_test, y_moons_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

In [ ]:
# ...or simpler way just use the grid search object directly
# (by default it refits the best estimator on the whole training set)
grid_dt.score(X_moons_test, y_moons_test)

### 8: Grow a forest

a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn's `ShuffleSplit` class for this.

In [ ]:
# Use ShuffleSplit to generate 1000 different training sets, each containing 100 instances
# selected randomly.
ss = ShuffleSplit(n_splits=1000, train_size=100, random_state=0)

subsets = []
for train_index, test_index in ss.split(X_moons_train):
    X_sub_train = X_moons_train[train_index]
    y_sub_train = y_moons_train[train_index]
    subsets.append((X_sub_train, y_sub_train))

b. Train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.

In [ ]:
# Train one Decision Tree on each subset, using the best hyperparameter values found above.
forest = [DecisionTreeClassifier(**grid_dt.best_params_, random_state=0) for _ in range(1000)]

for dt, (X_sub_train, y_sub_train) in zip(forest, subsets):
    dt.fit(X_sub_train, y_sub_train)

In [ ]:
# Now let's evaluate all these Decision Trees on the test set
forest_scores = [dt.score(X_moons_test, y_moons_test) for dt in dts]

In [ ]:
# As expected, mean score is about 80%
np.mean(forest_scores)

c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy's `mode()` function for this). This gives you _majority-vote predictions_ over the test set.

In [ ]:
for dt in forest:
    y_forest_pred = dt.predict(X_moons_test)